In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM
import tensorflow as tf
from sklearn.model_selection import train_test_split

c:\Users\Kalea\miniconda3\envs\tf_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Preparation

In [2]:
df = pd.read_csv('../Data/STAGE 4 FINAL MERGED DATA/STAGE_4_MERGED_FINAL.csv')

In [3]:
df

,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,total_pidana_penjara_bulan
0,Kebakaran Hutan,"Dewa Gde Ari Wicaksana, S,H.",RATIH KUSUMA WARDHANI,11,1 (satu) buah korek api kayu merk Three Durian...,"Bahwa Terdakwa I KETUT LANDUH, pada hari Selas...",I KETUT LANDUH,3.0
1,Penipuan,"FEBRI EKA PRADANA, S.H.",RATIH KUSUMA WARDHANI,4,1 (satu) lembar nota pembelian perhiasan kalun...,Bahwa terdakwa IDA BAGUS MADE DARMA WIGUNA ali...,IDA BAGUS MADE DARMA WIGUNA,24.0
2,Narkotika,"Gde Doni, S.H.",RATIH KUSUMA WARDHANI,2,1 (satu) buah plastik klip bening yang berisi ...,"Bahwa terdakwa I GEDE ARIADI alias BERNAD, pad...",I GEDE ARIADI alias BERNAD,14.0
3,Narkotika,"I Gde Doni Hendrawan, S.H.",RATIH KUSUMA WARDHANI,2,1 (satu) buah plastik klip bening yang berisi ...,"Bahwa terdakwa I GEDE ARIADI alias BERNAD, pad...",I GEDE ARIADI alias BERNAD,14.0
4,Penipuan,"Dewa Gde Ari Wicaksana, S,H.",ANAK AGUNG AYU DIAH INDRAWATI,5,1 (satu) unit sepeda motor merek Yamaha N-MAX ...,Bahwa Terdakwa I I KETUT JONI ADNYANA ADI PUTR...,I KETUT JONI ADNYANA ADI PUTRA,16.0
...,...,...,...,...,...,...,...,...
8577,Pencurian,"TRI NURANDI SINAGA, SH",SLAMET WIDODO,2,1 (satu) unit Handphone Samsung A52 Dikembalik...,"-Bahwa mereka, Terdakwa I SUPRIADI bin RATIM d...",ANWAR RURI BIN SYARIFUDIN USMAN,16.0
8578,Pencurian,"ARIF SURYANA , SH",DIAN ERDIANTO,2,"1 (satu) unit sepeda motor Yamaha NMAX, nopol ...",Perbuatan para terdakwa sebagaimana diatur dan...,BURHANNUDIN alias CULIK Bin M RUSDI,14.0
8579,Pencurian,"TRI NURANDI SINAGA, SH",DIAN ERDIANTO,2,"1 (satu) unit sepeda motor Yamaha NMAX, nopol ...",Perbuatan para terdakwa sebagaimana diatur dan...,BURHANNUDIN alias CULIK Bin M RUSDI,14.0
8580,Pencurian,"ARIF SURYANA , SH",DIAN ERDIANTO,2,"1 (satu) unit sepeda motor Yamaha NMAX, nopol ...",Perbuatan para terdakwa sebagaimana diatur dan...,DARWIS ABDUL SALAM alias AWIS Bin ALIMAN,14.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8582 entries, 0 to 8581
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   klasifikasi_perkara         8582 non-null   object 
 1   penuntut_umum               8582 non-null   object 
 2   hakim                       8582 non-null   object 
 3   jumlah_saksi                8582 non-null   int64  
 4   cleaned_barang_bukti        8582 non-null   object 
 5   cleaned_dakwaan             8582 non-null   object 
 6   terdakwa                    8582 non-null   object 
 7   total_pidana_penjara_bulan  8582 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 536.5+ KB


In [4]:
jenis_klasifikasi_perkara = df['klasifikasi_perkara'].unique()
jumlah_klasifikasi_perkara = len(jenis_klasifikasi_perkara)
print(f"Jumlah Klasifikasi Perkara: {jumlah_klasifikasi_perkara}")

klasifikasi_perkara_values = df['klasifikasi_perkara'].value_counts()
print(klasifikasi_perkara_values)

Jumlah Klasifikasi Perkara: 54
klasifikasi_perkara
Pencurian                                                                                               2656
Narkotika                                                                                               2528
Kejahatan Perjudian                                                                                      571
Penggelapan                                                                                              553
Penipuan                                                                                                 323
Penganiayaan                                                                                             270
Lalu Lintas                                                                                              249
Lain-Lain                                                                                                247
Tindak Pidana Senjata Api atau Benda Tajam                                   

### Filter 5 Klasifikasi Perkara Terbanyak

In [5]:
klasifikasi_perkara_terbanyak = klasifikasi_perkara_values.head(5).index
df_filtered = df[df['klasifikasi_perkara'].isin(klasifikasi_perkara_terbanyak)]

In [6]:
df_filtered

,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,total_pidana_penjara_bulan
1,Penipuan,"FEBRI EKA PRADANA, S.H.",RATIH KUSUMA WARDHANI,4,1 (satu) lembar nota pembelian perhiasan kalun...,Bahwa terdakwa IDA BAGUS MADE DARMA WIGUNA ali...,IDA BAGUS MADE DARMA WIGUNA,24.0
2,Narkotika,"Gde Doni, S.H.",RATIH KUSUMA WARDHANI,2,1 (satu) buah plastik klip bening yang berisi ...,"Bahwa terdakwa I GEDE ARIADI alias BERNAD, pad...",I GEDE ARIADI alias BERNAD,14.0
3,Narkotika,"I Gde Doni Hendrawan, S.H.",RATIH KUSUMA WARDHANI,2,1 (satu) buah plastik klip bening yang berisi ...,"Bahwa terdakwa I GEDE ARIADI alias BERNAD, pad...",I GEDE ARIADI alias BERNAD,14.0
4,Penipuan,"Dewa Gde Ari Wicaksana, S,H.",ANAK AGUNG AYU DIAH INDRAWATI,5,1 (satu) unit sepeda motor merek Yamaha N-MAX ...,Bahwa Terdakwa I I KETUT JONI ADNYANA ADI PUTR...,I KETUT JONI ADNYANA ADI PUTRA,16.0
5,Penipuan,"INDAH NOVITASARI, SH",ANAK AGUNG AYU DIAH INDRAWATI,5,1 (satu) unit sepeda motor merek Yamaha N-MAX ...,Bahwa Terdakwa I I KETUT JONI ADNYANA ADI PUTR...,I KETUT JONI ADNYANA ADI PUTRA,16.0
...,...,...,...,...,...,...,...,...
8577,Pencurian,"TRI NURANDI SINAGA, SH",SLAMET WIDODO,2,1 (satu) unit Handphone Samsung A52 Dikembalik...,"-Bahwa mereka, Terdakwa I SUPRIADI bin RATIM d...",ANWAR RURI BIN SYARIFUDIN USMAN,16.0
8578,Pencurian,"ARIF SURYANA , SH",DIAN ERDIANTO,2,"1 (satu) unit sepeda motor Yamaha NMAX, nopol ...",Perbuatan para terdakwa sebagaimana diatur dan...,BURHANNUDIN alias CULIK Bin M RUSDI,14.0
8579,Pencurian,"TRI NURANDI SINAGA, SH",DIAN ERDIANTO,2,"1 (satu) unit sepeda motor Yamaha NMAX, nopol ...",Perbuatan para terdakwa sebagaimana diatur dan...,BURHANNUDIN alias CULIK Bin M RUSDI,14.0
8580,Pencurian,"ARIF SURYANA , SH",DIAN ERDIANTO,2,"1 (satu) unit sepeda motor Yamaha NMAX, nopol ...",Perbuatan para terdakwa sebagaimana diatur dan...,DARWIS ABDUL SALAM alias AWIS Bin ALIMAN,14.0


### Split Data

In [28]:
def stratified_split(df: pd.DataFrame, split_size: float = 0.7) -> tuple[pd.DataFrame, pd.DataFrame]:
    unique_groups = df.groupby(["klasifikasi_perkara", "penuntut_umum", "hakim"]).apply(lambda x: x.index.tolist()).to_dict()
    
    print(unique_groups)
    train_idx, test_idx = [], []
    
    for indices in unique_groups.values():
        if len(indices) == 1:
            train_idx.extend(indices)
        else:
            train, test = train_test_split(indices, train_size=split_size, random_state=42)
            train_idx.extend(train)
            test_idx.extend(test)
    
    return df.loc[train_idx], df.loc[test_idx]

In [29]:
train_df, test_df = stratified_split(df_filtered)

C:\Users\Kalea\AppData\Local\Temp\ipykernel_3060\2311629469.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique_groups = df.groupby(["klasifikasi_perkara", "penuntut_umum", "hakim"]).apply(lambda x: x.index.tolist()).to_dict()


{('Kejahatan Perjudian', 'ADISTI PRATAMA FEREVALDY,SH', 'Ayu Putri Cempaka Sari'): [2907], ('Kejahatan Perjudian', 'ADISTI PRATAMA FEREVALDY,SH', 'Luh Sasmita Dewi'): [2757], ('Kejahatan Perjudian', 'ADISTI PRATAMA FEREVALDY,SH', 'Putu Gde Novyartha'): [2802, 2804], ('Kejahatan Perjudian', 'AKHIRUDIN VAMI KEMALSA, SH', 'R.R. Diah Poernomojekti'): [3666], ('Kejahatan Perjudian', 'ANAK AGUNG ALIT RAI SUASTIKA,SH.', 'Erwin Harlond Palyama'): [1541, 1542], ('Kejahatan Perjudian', 'ANAK AGUNG GEDE HENDRAWAN, SH.', 'Ayu Putri Cempaka Sari'): [2908], ('Kejahatan Perjudian', "ANDRIAN AL MAS'UDI, SH.MH.", 'DIAN ERDIANTO'): [6748], ('Kejahatan Perjudian', "ANDRIAN AL MAS'UDI, SH.MH.", 'HARTO PANCONO'): [6332], ('Kejahatan Perjudian', "ANDRIAN AL MAS'UDI, SH.MH.", 'I WAYAN GEDE RUMEGA'): [6746], ('Kejahatan Perjudian', 'ANINDITYA EKA BINTARI, SH. MH', 'NI MADE OKTIMANDIANI, SH'): [2226], ('Kejahatan Perjudian', 'ANINDITYA EKA BINTARI, SH. MH', 'PUTU ENDRU SONATA, SH, MH.'): [2247], ('Kejahatan Pe

In [27]:
jenis_klasifikasi_perkara = train_df['klasifikasi_perkara'].unique()
jumlah_klasifikasi_perkara = len(jenis_klasifikasi_perkara)
print(f"Jumlah Klasifikasi Perkara: {jumlah_klasifikasi_perkara}")
print(f"Jumlah Data Train:  {len(train_df)}")

klasifikasi_perkara_values = train_df['klasifikasi_perkara'].value_counts()
print(klasifikasi_perkara_values)

Jumlah Klasifikasi Perkara: 5
Jumlah Data Train:  4732
klasifikasi_perkara
Pencurian              1802
Narkotika              1779
Penggelapan             457
Kejahatan Perjudian     418
Penipuan                276
Name: count, dtype: int64


In [ ]:
jenis_klasifikasi_perkara = test_df['klasifikasi_perkara'].unique()
jumlah_klasifikasi_perkara = len(jenis_klasifikasi_perkara)
print(f"Jumlah Klasifikasi Perkara: {jumlah_klasifikasi_perkara}")
print(f"Jumlah Data Test:  {len(test_df)}")

klasifikasi_perkara_values = test_df['klasifikasi_perkara'].value_counts()
print(klasifikasi_perkara_values)

Jumlah Klasifikasi Perkara: 5
Jumlah Data Test:  1899
klasifikasi_perkara
Pencurian              854
Narkotika              749
Kejahatan Perjudian    153
Penggelapan             96
Penipuan                47
Name: count, dtype: int64


In [12]:
df_filtered

,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,total_pidana_penjara_bulan
1,Penipuan,"FEBRI EKA PRADANA, S.H.",RATIH KUSUMA WARDHANI,4,1 (satu) lembar nota pembelian perhiasan kalun...,Bahwa terdakwa IDA BAGUS MADE DARMA WIGUNA ali...,IDA BAGUS MADE DARMA WIGUNA,24.0
2,Narkotika,"Gde Doni, S.H.",RATIH KUSUMA WARDHANI,2,1 (satu) buah plastik klip bening yang berisi ...,"Bahwa terdakwa I GEDE ARIADI alias BERNAD, pad...",I GEDE ARIADI alias BERNAD,14.0
3,Narkotika,"I Gde Doni Hendrawan, S.H.",RATIH KUSUMA WARDHANI,2,1 (satu) buah plastik klip bening yang berisi ...,"Bahwa terdakwa I GEDE ARIADI alias BERNAD, pad...",I GEDE ARIADI alias BERNAD,14.0
4,Penipuan,"Dewa Gde Ari Wicaksana, S,H.",ANAK AGUNG AYU DIAH INDRAWATI,5,1 (satu) unit sepeda motor merek Yamaha N-MAX ...,Bahwa Terdakwa I I KETUT JONI ADNYANA ADI PUTR...,I KETUT JONI ADNYANA ADI PUTRA,16.0
5,Penipuan,"INDAH NOVITASARI, SH",ANAK AGUNG AYU DIAH INDRAWATI,5,1 (satu) unit sepeda motor merek Yamaha N-MAX ...,Bahwa Terdakwa I I KETUT JONI ADNYANA ADI PUTR...,I KETUT JONI ADNYANA ADI PUTRA,16.0
...,...,...,...,...,...,...,...,...
8577,Pencurian,"TRI NURANDI SINAGA, SH",SLAMET WIDODO,2,1 (satu) unit Handphone Samsung A52 Dikembalik...,"-Bahwa mereka, Terdakwa I SUPRIADI bin RATIM d...",ANWAR RURI BIN SYARIFUDIN USMAN,16.0
8578,Pencurian,"ARIF SURYANA , SH",DIAN ERDIANTO,2,"1 (satu) unit sepeda motor Yamaha NMAX, nopol ...",Perbuatan para terdakwa sebagaimana diatur dan...,BURHANNUDIN alias CULIK Bin M RUSDI,14.0
8579,Pencurian,"TRI NURANDI SINAGA, SH",DIAN ERDIANTO,2,"1 (satu) unit sepeda motor Yamaha NMAX, nopol ...",Perbuatan para terdakwa sebagaimana diatur dan...,BURHANNUDIN alias CULIK Bin M RUSDI,14.0
8580,Pencurian,"ARIF SURYANA , SH",DIAN ERDIANTO,2,"1 (satu) unit sepeda motor Yamaha NMAX, nopol ...",Perbuatan para terdakwa sebagaimana diatur dan...,DARWIS ABDUL SALAM alias AWIS Bin ALIMAN,14.0


## Model

### Model Preprocessing

In [13]:
train_df['concat_text'] = train_df[['klasifikasi_perkara', 'penuntut_umum', 'hakim', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'terdakwa']].apply(lambda x: ' '.join(x), axis=1)
test_df['concat_text'] = test_df[['klasifikasi_perkara', 'penuntut_umum', 'hakim', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'terdakwa']].apply(lambda x: ' '.join(x), axis=1)

In [14]:
y_train = train_df['total_pidana_penjara_bulan']
y_test = test_df['total_pidana_penjara_bulan']

### Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")

def tokenize_function(text):
    return tokenizer(text['concat_text'], padding='max_length', truncation=True, max_length=512)

train_encodings = tokenizer(train_df['concat_text'].tolist(), padding=True, truncation=True, max_length=512)
test_encodings = tokenizer(test_df['concat_text'].tolist(), padding=True, truncation=True, max_length=512)

### Convert to Tensor

In [ ]:
train_inputs = tf.constant(train_encodings['input_ids'])
train_attention_masks = tf.constant(train_encodings['attention_mask'])

test_inputs = tf.constant(test_encodings['input_ids'])
test_attention_masks = tf.constant(test_encodings['attention_mask'])

train_numerical = tf.constant(train_df['jumlah_saksi'].values, dtype=tf.float32)
test_numerical = tf.constant(test_df['jumlah_saksi'].values, dtype=tf.float32)

train_labels = tf.constant(y_train.values, dtype=tf.float32)
test_labels = tf.constant(y_test.values, dtype=tf.float32)

### Create Dataset from Tensor

In [ ]:
class BERTRegressorDataset(tf.data.Dataset):
    def __new__(self, input_ids, attention_mask, numerical_features, labels):
        dataset = tf.data.Dataset.from_tensor_slices(({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'numerical_feature': numerical_features
        }, labels))
        return dataset

train_dataset = BERTRegressorDataset(train_inputs, train_attention_masks, train_numerical, train_labels)
test_dataset = BERTRegressorDataset(test_inputs, test_attention_masks, test_numerical, test_labels)

train_dataset = train_dataset.batch(2)
test_dataset = test_dataset.batch(2)


### Model Architecture

In [ ]:
from transformers import TFBertModel
import tensorflow as tf

class BERTRegressor(tf.keras.Model):
    def __init__(self):
        super(BERTRegressor, self).__init__()
        self.bert = TFBertModel.from_pretrained("indolem/indobert-base-uncased", from_pt=True)
        self.regressor = tf.keras.layers.Dense(1)

    def call(self, inputs):
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        numerical_features = inputs['numerical_feature']

        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_output.pooler_output

        numerical_features = tf.expand_dims(numerical_features, axis=-1)
        combined_output = tf.concat([pooled_output, numerical_features], axis=1)

        return self.regressor(combined_output)

model = BERTRegressor()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predict

### Compile Model

In [24]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='mean_squared_error')

### Train Model

In [ ]:
history = model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Epoch 1/3


2366/2366 [==============================] - 410s 169ms/step - loss: 790.7148 - val_loss: 1038.7645
Epoch 2/3
2366/2366 [==============================] - 399s 169ms/step - loss: 668.5133 - val_loss: 908.3850
Epoch 3/3
2366/2366 [==============================] - 399s 169ms/step - loss: 584.2737 - val_loss: 529.8844


In [ ]:
tf.keras.backend.clear_session()

In [ ]:
import gc
gc.collect()


516

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU available:", gpus)
else:
    print("No GPU available, using CPU instead.")

GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
train_df = train_concat_text_df.reset_index(drop=True)
test_df = test_concat_text_df.reset_index(drop=True)

train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

In [ ]:
dataset_dict = DatasetDict({
    'train': train_data,
    'test': test_data
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['concat_text'],
        num_rows: 4732
    })
    test: Dataset({
        features: ['concat_text'],
        num_rows: 1899
    })
})


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
def tokenize(batch):
    return tokenizer(batch["concat_text"], padding=True, truncation=True, max_length=512)

In [ ]:
dataset_encoded = dataset_dict.map(tokenize, batched=True, batch_size=None)

Map: 100%|██████████| 1899/1899 [00:03<00:00, 583.76 examples/s]


In [ ]:
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['concat_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4732
    })
    test: Dataset({
        features: ['concat_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1899
    })
})

In [ ]:
dataset_encoded.set_format('tf', columns=['input_ids', 'attention_mask', 'token_type_ids', 'total_pidana_penjara_bulan'])

In [ ]:
def order(inp):
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]


In [ ]:
BATCH_SIZE = 16

train_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['train'][:])
test_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['test'][:])

# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(16, 512), dtype=int32, numpy=
array([[  101, 17710, 18878, ...,  4017,  2009,   102],
       [  101, 17710, 18878, ..., 26957,  2319,   102],
       [  101, 17710, 18878, ...,  4502,  2273,   102],
       ...,
       [  101, 17710, 18878, ...,  2226, 11333,   102],
       [  101, 17710, 18878, ...,  1012,  3156,   102],
       [  101, 17710, 18878, ...,  3366, 23630,   102]])>, 'attention_mask': <tf.Tensor: shape=(16, 512), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(16, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])>} 

 tf.Tensor(
[b'Kejahatan Perjudian A

In [ ]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras import layers


In [ ]:
from transformers import TFAutoModel

bert_model = TFAutoModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras import layers

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Define the regression model
def create_bert_regression_model():
    input_ids = layers.Input(shape=(512,), dtype=tf.int32, name="input_ids")  # Named input for input_ids
    attention_mask = layers.Input(shape=(512,), dtype=tf.int32, name="attention_mask")  # Named input for attention_mask

    # BERT layer
    bert_output = bert_model(input_ids, attention_mask=attention_mask)[0]  # Last hidden state
    pooler_output = bert_output[:, 0]  # Take [CLS] token representation for regression

    # Add a dense layer for regression
    output = layers.Dense(1, activation='linear')(pooler_output)  # Linear activation for regression

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss='mse')

    return model

# Create the model
model = create_bert_regression_model()
model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model_5 (TFBertModel)  TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                         

In [ ]:
history = model.fit(
    train_dataset,
    epochs=5
)

Epoch 1/5


c:\Users\Kalea\miniconda3\envs\tf_gpu\lib\site-packages\keras\engine\functional.py:637: UserWarning: Input dict contained keys ['token_type_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


UnimplementedError: Graph execution error:

Detected at node 'mean_squared_error/Cast' defined at (most recent call last):
    File "c:\Users\Kalea\miniconda3\envs\tf_gpu\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Kalea\miniconda3\envs\tf_gpu\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Kalea\miniconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\Kalea\miniconda3\envs\tf_gpu\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\Kalea\miniconda3\envs\tf_gpu\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\Kalea\miniconda3\envs\tf_gpu\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Kalea\miniconda3\

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('/content/drive/MyDrive/SpotifyEmotionRecognition_BERT/model_accuracy.png')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('/content/drive/MyDrive/SpotifyEmotionRecognition_BERT/model_loss.png')
plt.show()